<a href="https://colab.research.google.com/github/oya163/bert-llm/blob/master/cyber_security_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition in Cyber Security Domain

This google colab is about named entity recognition in cyber security domain by fine-tuning BERT-derivative model with the [CyNER](https://github.com/aiforsec/CyNER) dataset. This dataset is related to the vulnerabilities, firmware and cyber security.

## Installation

In [ ]:
!python3 -m pip install -U huggingface_hub
!python3 -m pip install -U accelerate
!python3 -m pip install -U transformers
!python3 -m pip install -U datasets evaluate
!python3 -m pip install -U seqeval

In [ ]:
# Wrap the text in ipython notebook
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Data Preprocessing

## Load MITRE (CyNER) dataset

Load MITRE dataset using our custom data loading script.


In [ ]:
from datasets import load_dataset

DATA_DIR = "/content/drive/MyDrive/cyber_security_dataset"

data_files = {
    "train": f"{DATA_DIR}/train.txt",
    "validation": f"{DATA_DIR}/valid.txt",
    "test": f"{DATA_DIR}/test.txt",
}

raw_datasets = load_dataset(f"{DATA_DIR}/load_ner.py", data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Check the basic information on the loaded dataset

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2811
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 813
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 748
    })
})

Check sample of tokens from train dataset

In [ ]:
raw_datasets["train"][0]["tokens"]

['Super',
 'Mario',
 'Run',
 'Malware',
 '#',
 '2',
 '–',
 'DroidJack',
 'RAT',
 'Gamers',
 'love',
 'Mario',
 'and',
 'Pokemon',
 ',',
 'but',
 'so',
 'do',
 'malware',
 'authors',
 '.']

Check the NER tags (its IDS) of the corresponding sample

In [ ]:
raw_datasets["train"][0]["ner_tags"]

[1, 2, 2, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-Malware', 'I-Malware', 'B-System', 'I-System', 'B-Organization', 'I-Organization', 'B-Indicator', 'I-Indicator', 'B-Vulnerability', 'I-Vulnerability'], id=None), length=-1, id=None)

Check the labels in the dataset

In [ ]:
label_names = ner_feature.feature.names
label_names

['O',
 'B-Malware',
 'I-Malware',
 'B-System',
 'I-System',
 'B-Organization',
 'I-Organization',
 'B-Indicator',
 'I-Indicator',
 'B-Vulnerability',
 'I-Vulnerability']

Display the token and labels

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

for x, y in zip(line1.split(), line2.split()):
    print(x, '\t', y)

Super 	 B-Malware
Mario 	 I-Malware
Run 	 I-Malware
Malware 	 I-Malware
# 	 O
2 	 O
– 	 O
DroidJack 	 B-Malware
RAT 	 I-Malware
Gamers 	 O
love 	 O
Mario 	 B-System
and 	 O
Pokemon 	 B-System
, 	 O
but 	 O
so 	 O
do 	 O
malware 	 O
authors 	 O
. 	 O


## Tokenization

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['<s>', '▁Super', '▁Mario', '▁Run', '▁Mal', 'ware', '▁#', '▁2', '▁–', '▁Dro', 'id', 'Jack', '▁', 'RAT', '▁Gam', 'ers', '▁love', '▁Mario', '▁and', '▁Pokemon', '▁', ',', '▁but', '▁so', '▁do', '▁malware', '▁author', 's', '▁', '.', '</s>']


In [ ]:
print(inputs.word_ids())

[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 7, 7, 8, 8, 9, 9, 10, 11, 12, 13, 14, 14, 15, 16, 17, 18, 19, 19, 20, 20, None]


## Data Preprocessing

In [ ]:
# Align the number of labels and the tokens
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[1, 2, 2, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0]
[-100, 1, 2, 2, 2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
# Helper function to tokenize and align labels
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
# Tokenize all the examples from the datasets
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/2811 [00:00<?, ? examples/s]

# Fine Tuning

## Data Collation

Prepare the dataloader for the training session

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    1,    2,    2,    2,    2,    0,    0,    0,    1,    2,    2,
            2,    2,    0,    0,    0,    3,    0,    3,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    3,
            1,    2,    0,    0,    0,    0,    0,    0,    0,    3,    4,    4,
            0,    0,    3,    0,    0, -100, -100]])

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 1, 2, 2, 2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1, 2, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 3, 0, 0, -100]


## Setup Evaluation

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("seqeval")


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config.num_labels

11

## Training

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.154353,0.612500,0.687500,0.647837,0.963656
2,0.203000,0.113410,0.697834,0.780612,0.736905,0.973082
3,0.061500,0.128796,0.752126,0.789541,0.770380,0.973746


TrainOutput(global_step=1056, training_loss=0.12833598572196384, metrics={'train_runtime': 826.4873, 'train_samples_per_second': 10.203, 'train_steps_per_second': 1.278, 'total_flos': 1373294813753520.0, 'train_loss': 0.12833598572196384, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.12879642844200134,
 'eval_precision': 0.7521263669501823,
 'eval_recall': 0.7895408163265306,
 'eval_f1': 0.7703795892968264,
 'eval_accuracy': 0.9737457668540307,
 'eval_runtime': 13.5242,
 'eval_samples_per_second': 60.114,
 'eval_steps_per_second': 7.542,
 'epoch': 3.0}

## Save the model

In [ ]:
model_path='/content/drive/MyDrive/bert_finetuned_ner/'
trainer.save_model(model_path)

## Evaluation

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])

In [ ]:
from tabulate import tabulate

metrics = ['precision', 'recall', 'f1', 'accuracy']
prediction_results = []

for key, val in predictions.metrics.items():
    if any(item in key for item in metrics):
        prediction_results.append([key, str(round(val,4)*100)+'%'])

print(tabulate(prediction_results, headers=['Metric', 'Score']))

Metric          Score
--------------  -------
test_precision  67.03%
test_recall     72.97%
test_f1         69.87%
test_accuracy   96.56%


## Inference

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/drive/MyDrive/bert_finetuned_ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
results = token_classifier("vulnerabilities reported BLU Products, founded in 2009, makes lower-end Android-powered smartphones that sell for as little as $50 on Amazon company.")

In [ ]:
prediction_results = []
for each_entity in results:
    prediction_results.append([each_entity['word'], each_entity['entity_group']])

print(tabulate(prediction_results, headers=['Word', 'Predictions']))


Word           Predictions
-------------  -------------
BLU Products   Organization
Android-power  System
Amazon         Organization


## Conclusion

After fine-tuning **xlm-roberta-large** model with **cyner** dataset, we achieved the following results

 * Precision : 67.26%
 * Recall : 73.00%
 * F1 : 69.87%
 * Accuracy : 96.56%